In [1]:
import torch 
bestMo = torch.load("resulttransMoco316CFAR002.pth")
dummy_input = torch.randn(256, 45, 256, 4, device='cuda')
bestMo.eval()

ModuleNotFoundError: No module named 'pytorch3d'

In [ ]:
torch.onnx.export(bestMo, dummy_input, "model.onnx", opset_version=11, export_params=True,
                  input_names=['radar_input'], output_names=['class_output', 'hidden_output']) 

In [3]:
import torch
import tvm.relay as relay
import tvm
import tvm.driver.tvmc as tvmc
from tvm.contrib import graph_executor
import numpy as np

# 1. Load and Prepare Model
bestMo = torch.load("resulttransMoco316CFAR002.pth")
bestMo.eval() 
dummy_input = torch.randn(256, 45, 256, 4, device='cuda')

# 2. ONNX Export
torch.onnx.export(bestMo, dummy_input, "model.onnx", opset_version=11, export_params=True,
                 input_names=['radar_input'], output_names=['class_output', 'hidden_output']) 

# 3 & 4. TVM Compilation and Runtime
target = "llvm"  # Or 'cuda' if you have a GPU
onnx_model = tvmc.load("model.onnx")
mod, params = relay.frontend.from_onnx(onnx_model)

with tvm.transform.PassContext(opt_level=3):
   lib = relay.build(mod, target=target, params=params)

dev = tvm.device(target, 0) 
module = graph_executor.GraphModule(lib["default"](dev))

input_data = tvm.nd.array(dummy_input.cpu().numpy()) 
module.set_input("radar_input", input_data) 
module.run()

# 5. Verification and Output
tvm_output_class = module.get_output(0).numpy()
tvm_output_hidden = module.get_output(1).numpy()


ModuleNotFoundError: No module named 'pytorch3d'